## 1. Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage
import warnings

from dataset_utils import load_dataset_split

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#warnings.filterwarnings('ignore')

In [3]:
ICOSEG_SUBSET_80_PATH = '../datasets/icoseg/subset_80'

## 2. Load val split

In [4]:
val_images, val_masks, img_names = load_dataset_split(ICOSEG_SUBSET_80_PATH, 'val')

mask = val_masks[0]
print(mask.shape, mask.dtype, mask.min(), mask.max())

(512, 512) float64 0.0 1.0


## 3. Jaccard similarity coefficient (JSC - also known as Intersection over Union)
The __*Jaccard index*__ can be expressed in terms of true positives (__*TP*__), false positives (__*FP*__) and false negatives (__FN__) as:  
__jaccard(A, B) = iou(A, B) = TP / (TP + FP + FN)__

In [5]:
def get_tp_fp_fn(a, b):
    a = np.equal(a, 1)
    not_a = np.logical_not(a)
    b = np.equal(b, 1)
    not_b = np.logical_not(b)
    
    tp = np.logical_and(a, b).sum().astype(np.float64)
    fp = np.logical_and(a, not_b).sum().astype(np.float64)
    fn = np.logical_and(not_a, b).sum().astype(np.float64)
    
    return tp, fp, fn

In [6]:
def jaccard(a, b):
    tp, fp, fn = get_tp_fp_fn(a, b)
    
    jaccard_coef = None
    if tp + fp + fn == 0:
        jaccard_coef = 1
    else:
        jaccard_coef = tp / (tp + fp + fn)
        
    return jaccard_coef

## 4. Dice similarity coefficient (DSC - also known as $F_1$-score)
The __*Dice coefficient*__ can be expressed in terms of true positives (__*TP*__), false positives (__*FP*__) and false negatives (__FN__) as:  
__dice(A, B) = 2TP / (2TP + FP + FN)__

In [7]:
def dice(a, b):
    tp, fp, fn = get_tp_fp_fn(a, b)
    
    dice_coef = None
    if tp + fp + fn == 0:
        dice_coef = 1
    else:
        dice_coef = (2 * tp) / (2 * tp + fp + fn)
    
    return dice_coef

## 5. Some tests

### Test 1

In [8]:
a = [[0, 0, 0],
     [0, 0, 0]]
b = [[0, 0, 0],
     [0, 0, 0]]
print(f'Jaccard: {jaccard(a, b)}, (expected: {1})')
print(f'Dice: {dice(a, b)}, (expected: {1})')

Jaccard: 1, (expected: 1)
Dice: 1, (expected: 1)


### Test 2

In [9]:
a = [[0, 0, 0],
     [0, 0, 0]]
b = [[1, 1, 1],
     [1, 1, 1]]
print(f'Jaccard: {jaccard(a, b)}, (expected: {0})')
print(f'Dice: {dice(a, b)}, (expected: {0})')

Jaccard: 0.0, (expected: 0)
Dice: 0.0, (expected: 0)


### Test 3

In [10]:
a = [[1, 1, 1],
     [1, 1, 1]]
b = [[0, 0, 0],
     [0, 0, 0]]
print(f'Jaccard: {jaccard(a, b)}, (expected: {0})')
print(f'Dice: {dice(a, b)}, (expected: {0})')

Jaccard: 0.0, (expected: 0)
Dice: 0.0, (expected: 0)


### Test 4

In [11]:
a = [[1, 1, 1],
     [1, 1, 1]]
b = [[1, 1, 1],
     [1, 1, 1]]
print(f'Jaccard: {jaccard(a, b)}, (expected: {1})')
print(f'Dice: {dice(a, b)}, (expected: {1})')

Jaccard: 1.0, (expected: 1)
Dice: 1.0, (expected: 1)


### Test 5

In [12]:
a = [[0, 1, 0],
     [0, 1, 0]]
b = [[0, 1, 0],
     [0, 1, 0]]
print(f'Jaccard: {jaccard(a, b)}, (expected: {1})')
print(f'Dice: {dice(a, b)}, (expected: {1})')

Jaccard: 1.0, (expected: 1)
Dice: 1.0, (expected: 1)


### Test 6

In [13]:
a = [[0, 1, 0],
     [0, 1, 0]]
b = [[0, 1, 0],
     [1, 1, 0]]
print(f'Jaccard: {jaccard(a, b)}, (expected: {2. / (2 + 0 + 1)})')
print(f'Dice: {dice(a, b)}, (expected: {2 * 2. / (2 * 2 + 0 + 1)})')

Jaccard: 0.6666666666666666, (expected: 0.6666666666666666)
Dice: 0.8, (expected: 0.8)


### Test 7

In [14]:
a = [[0, 1, 0],
     [1, 1, 0]]
b = [[0, 1, 0],
     [0, 1, 0]]
print(f'Jaccard: {jaccard(a, b)}, (expected: {2. / (2 + 1 + 0)})')
print(f'Dice: {dice(a, b)}, (expected: {2 * 2. / (2 * 2 + 1 + 0)})')

Jaccard: 0.6666666666666666, (expected: 0.6666666666666666)
Dice: 0.8, (expected: 0.8)
